# Домашнее задание №1
## Знакомство с языком обработки данных SAS Base
### Библиотеки и наборы данных SAS. Работа с переменными. Выборка данных. Вывод результатов в несколько наборов SAS

_Осипова Маргарита, Цветков Егор, Солонин Михаил_

### 0. Подготовка файлов для выполнения ДЗ.
Вместе с данным файлом в архиве находятся данные для выполнения ДЗ: 
- 4 набора данных SAS: **country**, **discount**, **employee_organization**, **orders**;
- 1 файл в формате excel (*.xlsx): **PRICES.XLSX***;

Скопируйте данные в директорию на вашем компьютере, которая связана с виртуальной машиной. Далее всю работу мы будем проводить в этой директории.


# 1. Создание библиотеки.
C помощью оператора libname создайте библиотеку SAS с именем *_Orion_*, которая "смотрит"/"является ссылкой" на директурию, в которую вы скопировали данные для выполнения задания.

In [1]:
LIBNAME Orion '/folders/myfolders/HW1';

### 2. Импорт файлов excel в SAS
Измените следующий код так, чтобы набор данных SAS (результат импорта из excel, который вы скопировали в п.0) помещался бы в библиотеку _Orion_.
Также напечатайте первые 5 наблюдений из полученного набора SAS.

In [4]:
proc import out=Orion.PRICES datafile='/folders/myfolders/HW1/PRICES.xlsx' dbms=XLSX replace; 
run;
proc print data=Orion.PRICES (obs=5); 
run;

Obs,Product_ID,Unit_Price,Factor
1,210200100009,$34.70,1.01
2,210200100017,$40.00,1.01
3,210200200023,$19.80,1.01
4,210200600067,$67.00,1.01
5,210200600085,$39.40,1.01


### 3.1 Оператор вывода наблюдений output;.
    Оператор output; немедленно выводит содержимое PVD в выходной набор данных. Если на Шаге DATA встретится несколько операторов output;, то для текущего наблюдения из входного набора будет создано несколько строк в выходном наборе (по количеству выполнившихся output;).
    Можно указать в какой набор выводить содержимое PVD: например, оператор output mylib.data; выводит в набор data в библиотеке mylib. Без указания набора содержимое сохраняется во все выходные наборы (все, что указаны в операторе data dataset1 dataset2 /*...*/ ;) 

Набор данных orion.prices содержит информацию о ценах на товары компнаии Orion Star.

###### a.   Напишите шаг DATA, создающий новый набор данных с прогнозами стоимости единицы товара на следующие три года. Этот набор данных должен содержать по три наблюдения на каждое наблюдение из входного набора данных orion.prices.
    - номер года (1,2 и 3) должен быть сохранен в переменной Year 
    - На основе набора orion.prices создаётся новый набор данных work.price_increase.  
    - Напишите явные операторы OUTPUT для вычисления прогнозов стоимости единицы товара (переменная unit_price) на три последующих года, причём коэффициент, задающий ежегодное изменение стоимости, хранится в переменной Factor. Прогноз здесь - это произведение Unit_Price предыдущего года и переменной Factor. Пример расчета для первого года приведен ниже.

In [8]:
data work.price_increase; 
    set orion.prices;
    
	do Year = 1 to 3;
	    Unit_Price=Unit_Price * Factor ** Year;
	    output;
	end;
    
run; 


##### b.   Напечатайте 6 наблюдений из полученного набора данных. 
    - Напечатайте только переменные Product_ID, Unit_Price и Year.   
    - Проверьте результаты. 


In [7]:
proc print data=work.price_increase (obs=6);    
    var Product_ID Unit_Price Year; 
run;

Obs,Product_ID,Unit_Price,Year
1,210200100009,$35.05,1
2,210200100009,$35.75,2
3,210200100009,$36.83,3
4,210200100017,$40.40,1
5,210200100017,$41.21,2
6,210200100017,$42.46,3


### 3.2  Создание нескольких наборов данных SAS 
    Набор данных orion.employee_organization содержит информацию о каждом сотруднике: должность (Job_title), подразделение (Department) и идентификатор менеджера (Manager_ID). 

#####   a.   Создайте отдельный набор данных для каждого подразделения. 
    Задайте следующие названия этих наборов данных: work.admin, work.stock и work.purchasing. 
    Используя условные операторы и явный оператор OUTPUT, выведите в эти наборы данных наблюдения, отвечающие подразделению Administration, Stock & Shipping или Purchasing, соответственно. Проигнорируйте все другие значения переменной Department.
   Подсказка: будьте аккуратны с регистром и написанием значений для Department. 

In [9]:
data work.admin work.stock work.purchasing;    
	set orion.employee_organization;
	if Department = "Administration" then output work.admin;
	if Department = "Stock & Shipping" then output work.stock;
	if Department = "Purchasing" then output work.purchasing;
run; 


Перепешите ваше решение, используя условный оператор select ...; when...; end...; или if, если только что уже использовали select.

In [10]:
data work.admin work.stock work.purchasing;
	set orion.employee_organization;
	
	select (Department);
	   when ("Administration") output work.admin;
	   when ("Stock & Shipping") output work.stock;
	   when ("Purchasing") output work.purchasing;
	   otherwise;
	end;
run;

### 4.1 Создание нескольких наборов данных SAS с вычисленными значениями 
Набор данных orion.orders содержит информацию о заказах, сделанных через магазины, по каталогу и в интернете, а также даты их доставки. 

##### a.   Необходимо подробнее изучить заказы по каталогу и через интернет, которые были доставлены быстро, а также с большими сроками доставки  
- Создайте три набора данных: work.fast, work.slow и  work.veryslow. 
- Напишите оператор WHERE, чтобы считать наблюдения, в которых Order_Type равен  2 (заказы по каталогу) или 3 (через интернет). 
- Создайте переменную ShipDays, содержащую число дней между датой совершения заказа (order_date) и датой доставки (delivery_date). 
- Обработайте выходные данные следующим образом: 
    – Выведите наблюдение в work.fast если значение ShipDays меньше чем 3. 
    – Выведите наблюдение в work.slow если значение ShipDays лежит в диапазоне от 5 до 7 (включительно). 
    – Выведите наблюдение в work.veryslow, если значение ShipDays превышает 7.
    – Не выводите наблюдение ни в один из наборов данных, если значение ShipDays равно 3 или 4.   
- В наборе данных work.fast должно быть 80 наблюдений, в work.slow - 69, а  в work.veryslow - 5. 
Замечание: Из 490 наблюдений во входном наборе данных orion.orders прочитаны только 230 (из-за оператора WHERE). 

In [11]:
data work.fast work.slow work.veryslow;
	set orion.orders;
	where Order_Type = 2 or Order_Type = 3;
	ShipDays = delivery_date - order_date;
	if ShipDays < 3 then output work.fast;
	if 5 <= ShipDays <= 7  then output work.slow;
	if ShipDays > 7 then output work.veryslow;

run;

### 5.1 Использование условной логики для вывода нескольких наблюдений 
Набор данных orion.country содержит информацию о полных названиях стран, а также различные коды для её поиска. 

##### a.   Создайте новый набор данных, содержащий по одному наблюдению на каждую страну, а также по одному наблюдению на каждое прежнее название этой страны (Country_FormerName).  
- Используя операторы условного выполнения if и явный оператор OUTPUT, создайте набор данных work.lookup. 
- Если для страны существует прежнее название, запишите два наблюдения, одно с текущим названием в переменной Country_Name, а другое - с прежним названием.
- Создайте новую переменную Outdated со значениями Y или N, в которой показывается, содержит ли это наблюдение актуальное название страны (Y) или нет (N).  

In [12]:
data work.lookup;
	set orion.country;
	Outdated = "Y";
	if not (Country_FormerName = " ") then
		do;
		output work.lookup;
		Country_Name = Country_FormerName;
		Country_FormerName = " ";
		Outdated = "N";
		output work.lookup;
		end;
	else output work.lookup;
	
run;